In [43]:
from __future__ import division, print_function, absolute_import
import tensorflow.compat.v1 as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior() 
import matplotlib.pyplot as plt
import numpy as np

from keras.utils import np_utils
from keras.models import Sequential

In [44]:
# citire imagini de antrenare plus etichete de antrenare
f = open ("train.txt")
train_images = []
train_labels = []
for line in enumerate(open("train.txt", "r")):
    v = line[1].split(',')
    image_name = 'train/'+v[0]
    label = int(v[1])
    train_labels.append(label)
    image = plt.imread(image_name)
    train_images.append(image)
    
train_images = np.expand_dims(train_images, axis=3)
train_images = np.array(train_images)
# train_labels = np.array(train_labels)


# citire imagini de testare 
f = open ("test.txt")
test_images = []
name_test_images =[]
for line in enumerate(open("test.txt", "r")):
    image_name = 'test/'+ line[1][:len(line[1])-1]
    name_test_images.append(line[1][:len(line[1])-1])
    image = plt.imread(image_name)
    test_images.append(image)
    
test_images = np.expand_dims(test_images, axis=3)

# citire imagini de validare plus etichete de validare
f = open ("validation.txt")
validation_images = []
validation_labels = []
for line in enumerate(open("validation.txt", "r")):
    v = line[1].split(',')
    image_name = 'validation/'+v[0]
    label = int(v[1])
    label = v[1]
    validation_labels.append(label)
    image = plt.imread(image_name)
    validation_images.append(image)

validation_images = np.expand_dims(validation_images, axis=3)
validation_labels = np.array(validation_labels)

In [45]:
n_classes = 9 #9 categorii notate de la 0 la 8

X_train=train_images
X_test=validation_images
test_images=test_images

Y_test=np_utils.to_categorical(validation_labels, n_classes) 
Y_train=np_utils.to_categorical(train_labels, n_classes)

In [46]:
from keras.applications import VGG16

# include top should be False to remove the softmax layer
pretrained_model = VGG16(include_top=False, weights='imagenet')
# pretrained_model.summary()

In [52]:
from keras.preprocessing.image import ImageDataGenerator

# create a new generator
imagegen = ImageDataGenerator()

# load train data
# train = imagegen.flow_from_directory("train/", class_mode="categorical", shuffle=False, batch_size=128, target_size=(224, 224))
train = imagegen.fit(train_images, train_labels)
# load val data
# val = imagegen.flow_from_directory("validation/", class_mode="categorical", shuffle=False, batch_size=128, target_size=(224, 224))

# train = imagegen.train(train_images, Y_train)

In [55]:
from keras.utils import to_categorical
# extract train and val features
vgg_features_train = pretrained_model.predict(train, verbose=1, steps=1)
# vgg_features_val = pretrained_model.predict(test_images)

ValueError: The two structures don't have the same nested structure.

First structure: type=NoneTensorSpec str=NoneTensorSpec()

Second structure: type=Tensor str=Tensor("input_4:0", shape=(?, ?, ?, 3), dtype=float32)

More specifically: Substructure "type=NoneTensorSpec str=NoneTensorSpec()" is a sequence, while substructure "type=Tensor str=Tensor("input_4:0", shape=(?, ?, ?, 3), dtype=float32)" is not
Entire first structure:
.
Entire second structure:
.

In [ ]:
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
model2 = Sequential()
model2.add(Flatten(input_shape=(7,7,512)))
model2.add(Dense(100, activation='relu'))
model2.add(Dropout(0.5))
model2.add(BatchNormalization())
model2.add(Dense(10, activation='softmax'))

# compile the model
model2.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

model2.summary()

# train model using features generated from VGG16 model
model2.fit(vgg_features_train, train_target, epochs=50, batch_size=128, validation_data=(vgg_features_val, val_target))